In [1]:
# GitHub update
!git add . && git commit -m "remove secret file" && git push

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ../../.gitignore
	deleted:    ../../Python of Algorithmic Trading/.ipynb_checkpoints/Py_4_AT-checkpoint.ipynb
	deleted:    ../../Python of Algorithmic Trading/Py_4_AT.ipynb
	deleted:    ../../Python of Algorithmic Trading/starter_files/001_equal_weight_S&P_500.ipynb
	deleted:    ../../Python of Algorithmic Trading/starter_files/002_quantitative_momentum_strategy.ipynb
	deleted:    ../../Python of Algorithmic Trading/starter_files/003_quantitative_value_strategy.ipynb
	deleted:    ../../Python of Algorithmic Trading/starter_files/__pycache__/secrets.cpython-39.pyc
	deleted:    ../../Python of Algorithmic Trading/starter_files/__pycache__/secrets_0x01.cpython-310.pyc
	deleted:    ../../Python of Algorithmic Trading/starter_files/__pycache__/secrets_0

# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
# %pip install -r requirements.txt

import numpy as np
import pandas as pd
import requests  # HTTP Requests for API interaction
import xlsxwriter
import math

So apparently, the tutorial is outdated due to IEX Cloud data provider having to change their policies over the years
and it is affecting the continuation of the course

To all those who are referring the course in 2024, following might help :


instead of IEX we can use yfinance library 


```
import yfinance as yf
tickerSymbol = 'AAPL'
tickerData = yf.Ticker(tickerSymbol)

# Get the historical prices for this ticker
tickerDf = tickerData.history(period='1d', start='2024-05-09', end='2024-05-10')
marketCap= tickerData.info['marketCap']


price= tickerDf['Open']

and also these prices are the live market value prices.
```

In [5]:
import yfinance as yf
from datetime import datetime, timedelta

ModuleNotFoundError: No module named 'yfinance'

**Now I am using Alpaca markets after realizing Yahoo Finance is flawed.**

In [5]:
# Please change the following to your own PAPER api key and secret
# You can get them from https://alpaca.markets/

API_KEY = "***********************"
SECRET_KEY = "***********************"

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

In [6]:
import json
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

import alpaca
from alpaca.data.live.stock import *
from alpaca.data.historical.stock import *
from alpaca.data.requests import *
from alpaca.data.timeframe import *
from alpaca.trading.client import *
from alpaca.trading.stream import *
from alpaca.trading.requests import *
from alpaca.trading.enums import *
from alpaca.common.exceptions import APIError


KeyboardInterrupt



In [25]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [26]:
# check version of alpaca-py
alpaca.__version__

'0.26.1'

#### Market Data (Historical)

In [28]:
import websocket, json

def on_open(ws):
    print("opened")
    auth_data = {
        "action": "authenticate",
        "data": {"key_id": secrets_0x01.API_KEY, "secret_key": secrets_0x01.SECRET_KEY}
    }

    ws.send(json.dumps(auth_data))

    listen_message = {"action": "listen", "data": {"streams": ["AM.TSLA"]}}

    ws.send(json.dumps(listen_message))


def on_message(ws, message):
    print("received a message")
    print(message)

def on_close(ws):
    print("closed connection")

socket = "wss://data.alpaca.markets/stream"

ws = websocket.WebSocketApp(socket, on_open=on_open, on_message=on_message, on_close=on_close)
ws.run_forever()

True

In [52]:
symbol = "TSLA"

In [7]:
# This is to test your Alpaca

import requests

url = "https://data.alpaca.markets/v2/stocks/auctions?symbols=AAPL%2CTSLA&limit=1000&sort=asc"

headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": "******************",
    "APCA-API-SECRET-KEY": "**************************"
}

response = requests.get(url, headers=headers)

print(response.text)

KeyboardInterrupt: 

In [54]:
from alpaca.data.timeframe import TimeFrame
from alpaca.data.requests import StockBarsRequest
from alpaca.data.historical import StockHistoricalDataClient# Create stock historical data client

client = StockHistoricalDataClient(API_KEY, SECRET_KEY)
request_params = StockBarsRequest(
                        symbol_or_symbols=["TSLA"],
                        timeframe=TimeFrame.Day,
                        start="2023-01-01 00:00:00",
                        end="2023-10-01 00:00:00",
                        feed=DataFeed.IEX,
                 )
bars = client.get_stock_bars(request_params)
bars_df = bars.df
print(bars_df)

                                     open     high      low    close  \
symbol timestamp                                                       
TSLA   2023-01-03 05:00:00+00:00  118.280  118.800  104.660  108.045   
       2023-01-04 05:00:00+00:00  109.150  114.570  107.620  113.660   
       2023-01-05 05:00:00+00:00  110.240  111.710  107.180  110.360   
       2023-01-06 05:00:00+00:00  102.900  114.380  101.870  113.010   
       2023-01-09 05:00:00+00:00  118.980  123.500  117.150  119.790   
...                                   ...      ...      ...      ...   
       2023-09-25 04:00:00+00:00  243.540  247.080  238.510  247.050   
       2023-09-26 04:00:00+00:00  242.955  249.510  241.735  244.240   
       2023-09-27 04:00:00+00:00  244.430  245.250  234.690  240.580   
       2023-09-28 04:00:00+00:00  240.240  247.450  238.750  246.335   
       2023-09-29 04:00:00+00:00  250.050  254.655  246.460  250.220   

                                     volume  trade_count       

In [47]:
# setup stock historical data client
stock_historical_data_client = StockHistoricalDataClient(API_KEY, SECRET_KEY, url_override = data_api_url)

In [55]:
# get historical bars by symbol
# ref. https://docs.alpaca.markets/reference/stockbars-1
now = datetime.now(ZoneInfo("America/New_York"))
req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Hour), # specify timeframe
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    limit = 2,                                               # specify limit
)
stock_historical_data_client.get_stock_bars(req).df

open    high     low     close  \
symbol timestamp                                                      
TSLA   2024-06-24 13:00:00+00:00  183.900  188.80  183.88  187.7300   
       2024-06-24 14:00:00+00:00  187.725  188.29  182.60  183.8614   

                                      volume  trade_count        vwap  
symbol timestamp                                                       
TSLA   2024-06-24 13:00:00+00:00  15387079.0     163461.0  186.955926  
       2024-06-24 14:00:00+00:00  15560327.0     212919.0  185.053644

In [58]:
# get historical trades by symbol
req = StockTradesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 2,                                                # specify limit
)
stock_historical_data_client.get_stock_trades(req).df

exchange   price  size    id  \
symbol timestamp                                                       
TSLA   2024-06-24 12:27:06.504934+00:00        D  184.00  25.0  1648   
       2024-06-24 12:27:07.218015+00:00        K  184.02   1.0  4054   

                                           conditions tape  
symbol timestamp                                            
TSLA   2024-06-24 12:27:06.504934+00:00     [@, T, I]    C  
       2024-06-24 12:27:07.218015+00:00  [@, F, T, I]    C

In [57]:
# get latest quotes by symbol
req = StockQuotesRequest(
    symbol_or_symbols = [symbol],
)
res = stock_historical_data_client.get_stock_latest_quote(req)
res

{'TSLA': {   'ask_exchange': 'V',
     'ask_price': 198.88,
     'ask_size': 1.0,
     'bid_exchange': 'V',
     'bid_price': 197.96,
     'bid_size': 2.0,
     'conditions': ['R'],
     'symbol': 'TSLA',
     'tape': 'C',
     'timestamp': datetime.datetime(2024, 6, 28, 19, 59, 59, 953084, tzinfo=TzInfo(UTC))}}

In [49]:
# It is for live streaming, do not run

# stock_data_stream_client = StockDataStream(API_KEY, SECRET_KEY, url_override = stream_data_wss)

# async def stock_data_stream_handler(data):
#     print(data)

# symbols = [symbol]

# stock_data_stream_client.subscribe_quotes(stock_data_stream_handler, *symbols) 
# stock_data_stream_client.subscribe_trades(stock_data_stream_handler, *symbols)

# stock_data_stream_client.run()

keyboard interrupt, bye


TimeoutError: 

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
type(stocks)
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [4]:
from secrets_0x01 import IEX_CLOUD_API_TOKEN, FMP_API_KEY #, API_KEY, SECRET_KEY

# there is no need for this anymore

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [5]:
#iex

symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
print(api_url)
data = requests.get(api_url)
# data = requests.get(api_url).json()
# print(data.status_code)
print(data)

In [70]:
#yfinance

symbol = 'AAPL'
data = yf.Ticker(symbol)

# Fetch the quote data
quote_data = data.info

# Get the historical prices for this ticker
tickerDf = data.history(period='1d', start='2020-06-25', end='2022-06-26')
# marketCap= data.info['marketCap']

price= tickerDf['Open']
# price = data.info['open']

# Print the quote data
print(quote_data)
# print(marketCap, price)

{'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '408 996 1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In add

In [77]:
#alpaca

symbol = 'AAPL'
api_url = f"https://data.alpaca.markets/v2/stocks/trades/latest?symbols={symbol}&feed=iex"
print(api_url)
data = requests.get(api_url, headers=headers).json()
# data = requests.get(api_url).json()
# print(data.status_code)
print(data)



https://data.alpaca.markets/v2/stocks/trades/latest?symbols=AAPL&feed=iex
{'trades': {'AAPL': {'c': ['@'], 'i': 12229, 'p': 210.61, 's': 200, 't': '2024-06-28T19:59:59.9707992Z', 'x': 'V', 'z': 'C'}}}


Stuck again, Alpaca is good for trading but not as detailed as IEX Cloud

In [6]:
#financial_modeling_prep

symbol = 'AAPL'
api_url = f'https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={FMP_API_KEY}'
print(api_url)

response = requests.get(api_url)
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Error fetching data:", response.status_code)

https://financialmodelingprep.com/api/v3/profile/AAPL?apikey=ftoIIveHDchl0jGuRcTyFw7sUzds4HMw
[{'symbol': 'AAPL', 'price': 220.11, 'beta': 1.24, 'volAvg': 62945703, 'mktCap': 3346574451000, 'lastDiv': 1, 'range': '164.08-237.23', 'changes': -0.8, 'companyName': 'Apple Inc.', 'currency': 'USD', 'cik': '0000320193', 'isin': 'US0378331005', 'cusip': '037833100', 'exchange': 'NASDAQ Global Select', 'exchangeShortName': 'NASDAQ', 'industry': 'Consumer Electronics', 'website': 'https://www.apple.com', 'description': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow cu

In [10]:
print(response.status_code)

200


For a more detailed example as to using `yfinance`

In [10]:
# tickerSymbol = 'AAPL'
# tickerData = yf.Ticker(tickerSymbol)

# # Get the current date and subtract one day to get a past date
# end_date = datetime.now() - timedelta(days=1)
# start_date = end_date - timedelta(days=1)

# # Format the dates as strings
# start_date_str = start_date.strftime('%Y-%m-%d')
# end_date_str = end_date.strftime('%Y-%m-%d')

# # Get the historical prices for this ticker
# tickerDf = tickerData.history(period='1d', start=start_date_str, end=end_date_str)

# # Fetch market cap
# marketCap = tickerData.info.get('marketCap', 'N/A')

# # Get the opening price
# price = tickerDf['Open'].iloc[0] if not tickerDf.empty else 'N/A'

# print(f"Market Cap: {marketCap}")
# print(f"Opening Price on {start_date_str}: {price}")

Market Cap: 3274597203968
Opening Price on 2024-06-25: 209.14999389648438


## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

> **_NOTE:_**  currentPrice is similar to what latestPrice is in IEX Cloud Data.

In [9]:
#yfinance

price = data.info['currentPrice']
market_cap = data.info['marketCap']
print(price, market_cap/1000000000000)

AttributeError: 'list' object has no attribute 'info'

In [7]:
#financial modeling prep

data = response.json()[0]
price = data['price']
market_cap = data['mktCap']
print(price, market_cap/1000000000000)

# if response.status_code == 200:
#     data = response.json()[0]
#     price = data['price']
#     print(price)
# else:
#     print("Error fetching data:", response.status_code)

220.11 3.346574451


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [8]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
# print(final_dataframe)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [9]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index=True
)

AttributeError: 'DataFrame' object has no attribute 'append'

The above won't run because the 'append' method in pandas has been deprecated and removed in recent versions

In [22]:
!python -c "import pandas; print(pandas.__version__)"

2.2.2


So make sure you do not end up like me and install the exact versions from the requirements.txt file. And it was not also clearly stated about the python version making some of the dependencies obsolete in the new version. But if I am to guess, I will state that it is Python 3.6.x

In [10]:
# Append a new row using pd.concat

# Initialise a pandas series object in prder to contain the data
# you want to append to the DataFrame
fd_append = pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index=my_columns
)

final_dataframe = pd.concat(
    [
        final_dataframe,
        fd_append.to_frame().T  #to_frame() -> converts the Series into a DF
                                #.T -> the transpose operation. converts the row into a column
    ], 
    ignore_index=True
)

final_dataframe


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,220.11,3346574451000,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [39]:
#financial_modeling_prep       
    
# Initialize an empty DataFrame with the specified columns
final_dataframe = pd.DataFrame(columns=my_columns)

for stock in stocks['Ticker'][:5]:
    api_url = f'https://financialmodelingprep.com/api/v3/profile/{stock}?apikey={FMP_API_KEY}'
    
    response = requests.get(api_url)
    
    if response.status_code == 200:
        data = response.json()[0]
        
        fd_append = pd.Series(
            [
                stock,
                data['price'],
                data['mktCap'],
                'N/A'
            ],
            index=my_columns
        )
        
        # Use pd.concat to append the new row as the append is deprecated
        final_dataframe = pd.concat(
            [
                final_dataframe,
                fd_append.to_frame().T
            ],
            ignore_index=True
        )
        
        # Uncomment the following line to print the current state of the DataFrame

        # print(f"Current state of final_dataframe:\n{final_dataframe}")
    else:
        print(f"Failed to retrieve data for stock: {stock}")

In [12]:
# Print the final DataFrame
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,138.03,39659883840,N/A
1,AAL,10.99,7217176960,N/A
2,AAP,38.53,2299250779,N/A
3,AAPL,220.11,3346574451000,N/A
4,ABBV,199.35,352119879000,N/A


The below is for YFinance and it is not as functional as FMP

In [ ]:
# %pip install --upgrade yfinance

In [40]:
final_dataframe = pd.DataFrame(columns=my_columns)

for stock in stocks['Ticker']:
    try:
        data = yf.Ticker(stock)
        stock_info = data.info
        
        # Check if the required keys are present in the info dictionary
        if 'currentPrice' in stock_info and 'marketCap' in stock_info:
            new_row = pd.DataFrame([[
                stock,
                stock_info['currentPrice'],
                stock_info['marketCap'],
                'N/A'
            ]], columns=my_columns)
            final_dataframe = pd.concat([final_dataframe, new_row], ignore_index=True)
        else:
            print(f"Missing data for {stock}: 'currentPrice' or 'marketCap' not found.")
    except Exception as e:
        print(f"Error fetching data for {stock}: {e}")

print(final_dataframe)

Error fetching data for A: name 'yf' is not defined
Error fetching data for AAL: name 'yf' is not defined
Error fetching data for AAP: name 'yf' is not defined
Error fetching data for AAPL: name 'yf' is not defined
Error fetching data for ABBV: name 'yf' is not defined
Error fetching data for ABC: name 'yf' is not defined
Error fetching data for ABMD: name 'yf' is not defined
Error fetching data for ABT: name 'yf' is not defined
Error fetching data for ACN: name 'yf' is not defined
Error fetching data for ADBE: name 'yf' is not defined
Error fetching data for ADI: name 'yf' is not defined
Error fetching data for ADM: name 'yf' is not defined
Error fetching data for ADP: name 'yf' is not defined
Error fetching data for ADSK: name 'yf' is not defined
Error fetching data for AEE: name 'yf' is not defined
Error fetching data for AEP: name 'yf' is not defined
Error fetching data for AES: name 'yf' is not defined
Error fetching data for AFL: name 'yf' is not defined
Error fetching data for A

As you can notice, there is an error that came up and this due to Yahoo API, I was getting it constantly and it was in the form of "404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PEAK?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=PEAK&crumb=2rPm363PiS" and when you click it, you get to see this ```"{"finance":{"result":null,"error":{"code":"Unauthorized","description":"Invalid Crumb"}}}```

This Stack Overflow came in handy and was able to fix it.

Update: yfinance still did not work

Note: Although I am looking into Alpaca API next as it is a somewhat intuitive and complete documentation for the Market Data API (free of course, no way I'm paying thousands for my personal projects hahaha).

In [ ]:
def get_yahoo_cookie():
    cookie = None

    user_agent_key = "User-Agent"
    user_agent_value = "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"

    headers = {user_agent_key: user_agent_value}
    response = requests.get(
        "https://fc.yahoo.com", headers=headers, allow_redirects=True
    )

    if not response.cookies:
        raise Exception("Failed to obtain Yahoo auth cookie.")

    cookie = list(response.cookies)[0]

    return cookie


def get_yahoo_crumb(cookie):
    crumb = None

    user_agent_key = "User-Agent"
    user_agent_value = "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"

    headers = {user_agent_key: user_agent_value}

    crumb_response = requests.get(
        "https://query1.finance.yahoo.com/v1/test/getcrumb",
        headers=headers,
        cookies={cookie.name: cookie.value},
        allow_redirects=True,
    )
    crumb = crumb_response.text

    if crumb is None:
        raise Exception("Failed to retrieve Yahoo crumb.")

    return crumb


# Usage
cookie = get_yahoo_cookie()
crumb = get_yahoo_crumb(cookie)

print(f"Here is the {cookie} and here is the {crumb}")

In [91]:
import requests
import pandas as pd
import yfinance as yf

def get_yahoo_cookie():
    user_agent_key = "User-Agent"
    user_agent_value = "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"

    headers = {user_agent_key: user_agent_value}
    response = requests.get("https://fc.yahoo.com", headers=headers, allow_redirects=True)

    if not response.cookies:
        raise Exception("Failed to obtain Yahoo auth cookie.")

    cookie = list(response.cookies)[0]
    return cookie

def get_yahoo_crumb(cookie):
    user_agent_key = "User-Agent"
    user_agent_value = "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"

    headers = {user_agent_key: user_agent_value}

    crumb_response = requests.get(
        "https://query1.finance.yahoo.com/v1/test/getcrumb",
        headers=headers,
        cookies={cookie.name: cookie.value},
        allow_redirects=True,
    )
    crumb = crumb_response.text

    if not crumb:
        raise Exception("Failed to retrieve Yahoo crumb.")

    return crumb

def fetch_stock_data(stock, cookie, crumb):
    user_agent_key = "User-Agent"
    user_agent_value = "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
    headers = {user_agent_key: user_agent_value}
    url = f"https://query2.finance.yahoo.com/v10/finance/quoteSummary/{stock}?modules=financialData,quoteType,defaultKeyStatistics,assetProfile,summaryDetail&crumb={crumb}"

    response = requests.get(url, headers=headers, cookies={cookie.name: cookie.value})
    if response.status_code != 200:
        raise Exception(f"Failed to fetch data for {stock}: {response.status_code} {response.reason}")
    
    return response.json()

def main():
    my_columns = ['Ticker', 'Current Price', 'Market Cap', 'Other Info']
    final_dataframe = pd.DataFrame(columns=my_columns)
    stocks = pd.DataFrame({'Ticker': ['ABC', 'ADI', 'ALXN', 'ANTM', 'ATVI', 'BF.B', 'BLL', 'BRK.B', 'CERN', 'COG', 'CTL', 'CTXS', 'CXO', 'DISCA', 'DISCK', 'DISH', 'DRE', 'ETFC', 'FB', 'FBHS', 'FISV', 'FLIR', 'FLT', 'FRC', 'HFC', 'INFO', 'KSU', 'MXIM', 'MYL', 'NBL', 'NLOK', 'NLSN', 'PBCT', 'PEAK']})

    cookie = get_yahoo_cookie()
    crumb = get_yahoo_crumb(cookie)

    for stock in stocks['Ticker']:
        try:
            data = fetch_stock_data(stock, cookie, crumb)
            stock_info = data['quoteSummary']['result'][0]
            
            if 'currentPrice' in stock_info['financialData'] and 'marketCap' in stock_info['defaultKeyStatistics']:
                new_row = pd.DataFrame([[
                    stock,
                    stock_info['financialData']['currentPrice']['raw'],
                    stock_info['defaultKeyStatistics']['marketCap']['raw'],
                    'N/A'
                ]], columns=my_columns)
                final_dataframe = pd.concat([final_dataframe, new_row], ignore_index=True)
            else:
                print(f"Missing data for {stock}: 'currentPrice' or 'marketCap' not found.")
        except requests.exceptions.HTTPError as e:
            print(f"HTTPError fetching data for {stock}: {e}")
        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")

    print(final_dataframe)


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=2rPm363PiS.


Missing data for ABC: 'currentPrice' or 'marketCap' not found.
RequestException fetching data for ADI: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out.
Missing data for ALXN: 'currentPrice' or 'marketCap' not found.
Missing data for ANTM: 'currentPrice' or 'marketCap' not found.


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ATVI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ATVI&crumb=2rPm363PiS.


Missing data for ATVI: 'currentPrice' or 'marketCap' not found.
Missing data for BF.B: 'currentPrice' or 'marketCap' not found.
Missing data for BLL: 'currentPrice' or 'marketCap' not found.
Missing data for BRK.B: 'currentPrice' or 'marketCap' not found.
Missing data for CERN: 'currentPrice' or 'marketCap' not found.
Missing data for COG: 'currentPrice' or 'marketCap' not found.
Missing data for CTL: 'currentPrice' or 'marketCap' not found.
Missing data for CTXS: 'currentPrice' or 'marketCap' not found.
Missing data for CXO: 'currentPrice' or 'marketCap' not found.
Missing data for DISCA: 'currentPrice' or 'marketCap' not found.
Missing data for DISCK: 'currentPrice' or 'marketCap' not found.


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DISH?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=DISH&crumb=2rPm363PiS.


...

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/RE?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=RE&crumb=2rPm363PiS.


Missing data for RE: 'currentPrice' or 'marketCap' not found.
RequestException fetching data for SEE: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
RequestException fetching data for SHW: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v10/finance/quoteSummary/SHW?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SHW&crumb=2rPm363PiS. (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000026B6E4E7280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
RequestException fetching data for SIVB: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v10/finance/quoteSummary/SIVB?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDom

RequestException fetching data for TMO: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v10/finance/quoteSummary/TMO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TMO&crumb=2rPm363PiS. (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000026B6E4E71C0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
RequestException fetching data for TMUS: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v10/finance/quoteSummary/TMUS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TMUS&crumb=2rPm363PiS. (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000026B6E4E76A0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

RequestException fetching data for VRSN: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v10/finance/quoteSummary/VRSN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=VRSN&crumb=2rPm363PiS. (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000026B6E4E7760>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
RequestException fetching data for VRTX: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v10/finance/quoteSummary/VRTX?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=VRTX&crumb=2rPm363PiS. (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000026B6E4E7070>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed

RequestException fetching data for ZION: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v10/finance/quoteSummary/ZION?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ZION&crumb=2rPm363PiS. (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000026B6E4E7940>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
RequestException fetching data for ZTS: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v10/finance/quoteSummary/ZTS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ZTS&crumb=2rPm363PiS. (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000026B6E4E7FD0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [41]:
# Gotten from stackoverflow
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

As it seems, FMP does not have a batch API call as a free tier version and we would need to pay. So, we would need to tweak our API to get the result we need.

In [42]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    # this is the profile endpoint API but lets stick with the name
    batch_api_call_url = f'https://financialmodelingprep.com/api/v3/profile/{symbol_string}?apikey={FMP_API_KEY}'
    
    response = requests.get(batch_api_call_url)
    
    if response.status_code == 200:
        data = response.json()
        
        # FMP Json data is different from IEX
        # for symbol in symbol_string.split(','):
        for info in data:
            fd_append = pd.Series(
                [
                    info['symbol'],
                    info['price'],
                    info['mktCap'],
                    'N/A'
                ],
                index=my_columns
            )

            final_dataframe = pd.concat(
                [
                    final_dataframe,
                    fd_append.to_frame().T  #to_frame() -> converts the Series into a DF
                                            #.T -> the transpose operation. converts the row into a column
                ], 
                ignore_index=True
            )
    # error handling
    else:
        print(f"Failed to retrieve profile data for symbols: {symbol_string}")
        
# Print the final DataFrame    
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,138.03,39659883840,N/A
1,AAL,10.99,7217176960,N/A
2,AAP,38.53,2299250779,N/A
3,AAPL,220.11,3346574451000,N/A
4,ABBV,199.35,352119879000,N/A
...,...,...,...,...
490,YUM,132.89,37364016850,N/A
491,ZBRA,334.85,17271696940,N/A
492,ZION,45.78,6761568660,N/A
493,ZBH,105.16,21416044320,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

Note: In the tutorial, we were told to assume the user won't try to input the wrong data type more than twice as that will still trigger the error so I fixed it up. We can use the `While` loop.

In [43]:
while True:
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
        print(val)
        break  # Exit the loop if input is valid
    except ValueError:
        print("That's not a number! \nPlease try again:")

Enter the value of your portfolio: 10000000


10000000.0


In [44]:
# Calculate position size
position_size = val / len(final_dataframe.index)
# print(position_size)

# Calculate the number of shares to buy
for i in range(0, len(final_dataframe.index)):
    stock_price = final_dataframe.loc[i, 'Stock Price']
    if pd.notna(stock_price):  # Check if stock_price is not NaN
        final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/stock_price)
    else:
        final_dataframe.loc[i, 'Number of Shares to Buy'] = 0  # or any other default value

# Print the final DataFrame
final_dataframe

# position_size = float(portfolio_size) / len(final_dataframe.index)
# for i in range(0, len(final_dataframe['Ticker'])-1):
#     final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
# final_dataframe


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,138.03,39659883840,146
1,AAL,10.99,7217176960,1838
2,AAP,38.53,2299250779,524
3,AAPL,220.11,3346574451000,91
4,ABBV,199.35,352119879000,101
...,...,...,...,...
490,YUM,132.89,37364016850,152
491,ZBRA,334.85,17271696940,60
492,ZION,45.78,6761568660,441
493,ZBH,105.16,21416044320,192


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [46]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(excel_writer=writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format (
    {
        font_color

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy:

## Yahoo Finance info

Here are the key fields present in the provided data:

1. **Company Information:**
   - `address1`: Company address.
   - `city`: City of the company.
   - `state`: State of the company.
   - `zip`: Zip code of the company.
   - `country`: Country of the company.
   - `phone`: Company phone number.
   - `website`: Company website URL.
   - `industry`: Industry the company operates in.
   - `industryKey`: Key for the industry.
   - `industryDisp`: Display name for the industry.
   - `sector`: Sector the company belongs to.
   - `sectorKey`: Key for the sector.
   - `sectorDisp`: Display name for the sector.
   - `longBusinessSummary`: Detailed summary of the company's business.
   - `fullTimeEmployees`: Number of full-time employees.
   - `companyOfficers`: List of company officers with details such as name, age, title, and pay.

2. **Risk Metrics:**
   - `auditRisk`: Risk associated with audit.
   - `boardRisk`: Risk associated with the board.
   - `compensationRisk`: Risk associated with compensation.
   - `shareHolderRightsRisk`: Risk associated with shareholder rights.
   - `overallRisk`: Overall risk assessment.

3. **Governance and Dates:**
   - `governanceEpochDate`: Epoch date for governance.
   - `compensationAsOfEpochDate`: Epoch date for compensation.
   - `irWebsite`: Investor relations website URL.
   - `maxAge`: Maximum age of the data.

4. **Stock Information:**
   - `priceHint`: Price hint for the stock.
   - `previousClose`: Previous closing price.
   - `open`: Opening price.
   - `dayLow`: Lowest price of the day.
   - `dayHigh`: Highest price of the day.
   - `regularMarketPreviousClose`: Previous closing price in regular market.
   - `regularMarketOpen`: Opening price in regular market.
   - `regularMarketDayLow`: Lowest price of the day in regular market.
   - `regularMarketDayHigh`: Highest price of the day in regular market.
   - `dividendRate`: Dividend rate.
   - `dividendYield`: Dividend yield.
   - `exDividendDate`: Ex-dividend date.
   - `payoutRatio`: Payout ratio.
   - `fiveYearAvgDividendYield`: Five-year average dividend yield.
   - `beta`: Beta value.
   - `trailingPE`: Trailing price-to-earnings ratio.
   - `forwardPE`: Forward price-to-earnings ratio.
   - `volume`: Trading volume.
   - `regularMarketVolume`: Trading volume in regular market.
   - `averageVolume`: Average trading volume.
   - `averageVolume10days`: Average trading volume over 10 days.
   - `averageDailyVolume10Day`: Average daily trading volume over 10 days.
   - `bid`: Bid price.
   - `ask`: Ask price.
   - `bidSize`: Bid size.
   - `askSize`: Ask size.
   - `marketCap`: Market capitalization.
   - `fiftyTwoWeekLow`: 52-week low price.
   - `fiftyTwoWeekHigh`: 52-week high price.
   - `priceToSalesTrailing12Months`: Price-to-sales ratio over the trailing 12 months.
   - `fiftyDayAverage`: 50-day moving average.
   - `twoHundredDayAverage`: 200-day moving average.
   - `trailingAnnualDividendRate`: Trailing annual dividend rate.
   - `trailingAnnualDividendYield`: Trailing annual dividend yield.
   - `currency`: Currency of the stock.
   - `enterpriseValue`: Enterprise value.
   - `profitMargins`: Profit margins.
   - `floatShares`: Number of shares available for trading.
   - `sharesOutstanding`: Number of shares outstanding.
   - `sharesShort`: Number of shares shorted.
   - `sharesShortPriorMonth`: Number of shares shorted in the prior month.
   - `sharesShortPreviousMonthDate`: Date of the previous month's short interest.
   - `dateShortInterest`: Date of the short interest.
   - `sharesPercentSharesOut`: Percentage of shares outstanding that are shorted.
   - `heldPercentInsiders`: Percentage of shares held by insiders.
   - `heldPercentInstitutions`: Percentage of shares held by institutions.
   - `shortRatio`: Short ratio.
   - `shortPercentOfFloat`: Percentage of float that is shorted.
   - `impliedSharesOutstanding`: Implied number of shares outstanding.
   - `bookValue`: Book value.
   - `priceToBook`: Price-to-book ratio.
   - `lastFiscalYearEnd`: Last fiscal year end date.
   - `nextFiscalYearEnd`: Next fiscal year end date.
   - `mostRecentQuarter`: Most recent quarter date.
   - `earningsQuarterlyGrowth`: Quarterly earnings growth.
   - `netIncomeToCommon`: Net income to common shareholders.
   - `trailingEps`: Trailing earnings per share.
   - `forwardEps`: Forward earnings per share.
   - `pegRatio`: Price/earnings to growth ratio.
   - `lastSplitFactor`: Last stock split factor.
   - `lastSplitDate`: Last stock split date.
   - `enterpriseToRevenue`: Enterprise value to revenue ratio.
   - `enterpriseToEbitda`: Enterprise value to EBITDA ratio.
   - `52WeekChange`: 52-week price change.
   - `SandP52WeekChange`: S&P 52-week price change.
   - `lastDividendValue`: Last dividend value.
   - `lastDividendDate`: Last dividend date.
   - `exchange`: Stock exchange.
   - `quoteType`: Type of quote.
   - `symbol`: Stock symbol.
   - `underlyingSymbol`: Underlying symbol.
   - `shortName`: Short name of the company.
   - `longName`: Long name of the company.
   - `firstTradeDateEpochUtc`: First trade date in epoch UTC.
   - `timeZoneFullName`: Full name of the time zone.
   - `timeZoneShortName`: Short name of the time zone.
   - `uuid`: Unique identifier.
   - `messageBoardId`: Message board ID.
   - `gmtOffSetMilliseconds`: GMT offset in milliseconds.
   - `currentPrice`: Current price.
   - `targetHighPrice`: Target high price.
   - `targetLowPrice`: Target low price.
   - `targetMeanPrice`: Target mean price.
   - `targetMedianPrice`: Target median price.
   - `recommendationMean`: Mean recommendation.
   - `recommendationKey`: Recommendation key.
   - `numberOfAnalystOpinions`: Number of analyst opinions.
   - `totalCash`: Total cash.
   - `totalCashPerShare`: Total cash per share.
   - `ebitda`: Earnings before interest, taxes, depreciation, and amortization.
   - `totalDebt`: Total debt.
   - `quickRatio`: Quick ratio.
   - `currentRatio`: Current ratio.
   - `totalRevenue`: Total revenue.
   - `debtToEquity`: Debt-to-equity ratio.
   - `revenuePerShare`: Revenue per share.
   - `returnOnAssets`: Return on assets.
   - `returnOnEquity`: Return on equity.
   - `freeCashflow`: Free cash flow.
   - `operatingCashflow`: Operating cash flow.
   - `earningsGrowth`: Earnings growth.
   - `revenueGrowth`: Revenue growth.
   - `grossMargins`: Gross margins.
   - `ebitdaMargins`: EBITDA margins.
   - `operatingMargins`: Operating margins.
   - `financialCurrency`: Financial currency.
   - `trailingPegRatio`: Trailing price/earnings to growth ratio.

## **Key Fields in Financial Modeling Prep's Company Profile API Response:**

**Company Identification and Basic Information:**

* **symbol:** The stock ticker symbol (e.g., AAPL, GOOGL, TSLA).
* **companyName:** The full legal name of the company.
* **currency:** The currency in which the stock is traded (e.g., USD, EUR, JPY).
* **cik:** Central Index Key, a unique identifier assigned to companies by the U.S. Securities and Exchange Commission (SEC).
* **isin:** International Securities Identification Number, a unique global identifier for securities.
* **cusip:** Committee on Uniform Security Identification Procedures number, a unique identifier for North American securities.
* **exchange:** The stock exchange where the company is listed (e.g., NASDAQ, NYSE, LSE).
* **exchangeShortName:** A shorter name for the exchange.
* **industry:** The industry sector the company belongs to (e.g., Technology, Healthcare, Finance).
* **website:** The company's official website.
* **description:** A brief overview of the company's business activities and products.
* **ceo:** The name of the Chief Executive Officer.
* **sector:** A broader classification of the company's industry.
* **country:** The country where the company is headquartered.
* **fullTimeEmployees:** The number of full-time employees.
* **phone:** The company's phone number.
* **address:** The company's headquarters address.
* **city, state, zip:** The city, state, and ZIP code of the headquarters (if applicable).
* **image:** A URL to the company's logo image.
* **ipoDate:** The date of the company's Initial Public Offering (IPO) (if applicable).
* **isEtf, isActivelyTrading, isAdr, isFund:** Boolean flags indicating whether the security is an ETF, actively traded, an ADR, or a fund.

**Financial and Market Data:**

* **price:** The current stock price.
* **beta:** A measure of the stock's volatility compared to the overall market. A beta of 1 means the stock moves in line with the market, while a beta greater than 1 indicates higher volatility, and less than 1 indicates lower volatility.
* **volAvg:** The average trading volume (number of shares traded per day).
* **mktCap:** The market capitalization, calculated as the stock price multiplied by the number of outstanding shares.
* **lastDiv:** The amount of the most recent dividend paid per share (if applicable).
* **range:** The 52-week price range (low-high).
* **changes:** The change in price from the previous day's close.

**Other Information:**

* **dcfDiff, dcf:** Discounted Cash Flow (DCF) related data. DCF is a valuation method that estimates the value of an investment based on its expected future cash flows.
* **defaultImage:** Indicates if the image is a default one or specific to the company.
* **governanceEpochDate, compensationAsOfEpochDate:** Dates related to corporate governance and executive compensation data.
* **irWebsite:** URL of the company's investor relations website.
* **maxAge:** Indicates how long the data is considered valid (usually in seconds).
